In [ ]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.feature import RegexTokenizer,CountVectorizer
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import StopWordsRemover
# sentenceDataFrame = spark.createDataFrame -- to create dataframe
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import NGram
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.classification import LogisticRegression
from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
import numpy as np
#Vector assembler , string indexer and index to string from spark ML
# spark.driver.maxResultSize
spark = SparkSession.builder.master("local[*]")\
        .config("spark.executor.memory", "16g")\
        .config("spark.driver.memory", "16g")\
        .config("spark.memory.offHeap.enabled",'true')\
        .config("spark.memory.offHeap.size","8g")\
        .getOrCreate()
sqlContext = SQLContext(spark)

In [ ]:
#load csv file (only for version 1.4 and above--current version is 2.4.5 BTW)
import pandas as pd
df = pd.read_csv('/Users/karunparashar/Downloads/Assignment 3 DIC/train2.csv')
dataframe = sqlContext.createDataFrame(df)
df2 = pd.read_csv('/Users/karunparashar/Downloads/Assignment 3 DIC/dic487-587/test.csv')
dataframe2 = sqlContext.createDataFrame(df2)

In [ ]:
add_stopwords = ["http","https","amp","rt","t","c","the",'br','find','one', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"] 
regexTokenizer2 = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
stopwordsRemover2 = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

Total time taken-35 minutes

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import HashingTF, IDF
mapping_dict = {'Drama':0,
               'Comedy':1,
               'Romance Film':2,
               'Thriller':3,
               'Action':4,
               'World cinema':5,
               'Crime Fiction':6,
               'Horror':7,
               'Black-and-white':8,
               'Indie':9,
               'Action/Adventure':10,
               'Adventure':11,
               'Family Film':12,
               'Short Film':13,
               'Romantic drama':14,
               'Animation':15,
               'Musical':16,
               'Science Fiction':17,
               'Mystery':18,
               'Romantic comedy':19}
nd_TFIDF ={}
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=500)
for i in mapping_dict:
    udf = UserDefinedFunction(lambda x: 1 if i in x else 0 , IntegerType())
    new_df = dataframe.withColumn('label', udf(dataframe.genre))
    pipeline = Pipeline(stages=[regexTokenizer2, stopwordsRemover2, hashingTF, idf])
    pipelineFit = pipeline.fit(new_df)
    dataset = pipelineFit.transform(new_df)
    dataframe_test_1 =pipelineFit.transform(dataframe2) 
    lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
    lrModel = lr.fit(dataset)
    predictions = lrModel.transform(dataframe_test_1)
    nd_TFIDF[i] = np.array(predictions.select('prediction').collect())

In [ ]:
#how the prediction stored for each genre looks like
for i in nd_TFIDF:
    nd_TFIDF[i] = list(nd_TFIDF[i])
nd_TFIDF

In [ ]:
#converting dictionary to dataframe for output compatible format store in CSV file,pandas has been used only for this purpose
a1 =pd.DataFrame(nd_TFIDF).astype(int)
predds = pd.Series(list(a1.iloc[i,]) for i in range(len(a1)))#to do in spark
pd.DataFrame(predds).to_csv('intermediate_TFIDF_logreg_16.csv',index=False)